# Lab 2 - Summarizing Network Attacks

# Overview
In this lab, we will try to predict malicious network attacks. 
There are many different types of attacks, but for this lab we
will only try to predict normal vs malicious.

We will need to import `pandas` and `numpy` for this lab

In [ ]:
import pandas as pd
import numpy as np

# Data
The data comes from 
[KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html).
The data contains several million network connections from a lab
environment. The purpose of this data was to help build a network
intrusion detector. 

The full dataset is very large, consisting of nearly 5 million rows. You may wish
to use the 10% subset of the data while writing your code to speed up
model training. Once your code is proper, you can remove the
subset and instead use the entire dataset.

Download the files `kddcup.data_10_percent.gz` and `kddcup.names` from the website.

## Data cleaning and preprocessing

The dataset does not have column names stored in the file, so we must use the `kddcup.names` file
to label the columns. This is our first task. The file can be treated as a `csv` by using 
`:` as the delimiter (instead of `,`). The first row contains a list of the malicious types 
of traffic, so we need to skip that. The last column in the dataset contains the type of attack (or if the traffic
is normal). We will manually add this to the end of our list using `.append()`.

In [ ]:
# load txt file
names = pd.read_csv('data/kddcup.names', header=None, delimiter=':',skiprows=1)

# make column 0 into a list
name_list = names[0].tolist()

# add the last column with type
name_list.append('type')

print(name_list)

Next load the data file and use the column names from the `name_list` list that we created.

In [ ]:
netattacks = pd.read_csv('data/kddcup.data_10_percent_corrected', names=name_list, header=None, index_col=None)

# Basic dataframe statistics
First, let's check the first five rows using the `.head()` function. You can also view the last five rows using the `.tail()` function.

In [ ]:
netattacks.head()

We will use the `.describe()` function to get a summary of the columns.

In [ ]:
netattacks.describe(include='all')

Printing this out directly will omit some columns for wide dataframes like this. If you want to see all of the columns, an easy way is to export the stats to a `csv` and open it with Excel.

In [ ]:
# store stats in a dataframe
df_stats = netattacks.describe(include='all')
# save dataframe to file
df_stats.to_csv('output/netattack_summary.csv')

# Summarize by type of attack
Now that we have loaded the data and things seem to be reasonable, let's look into the types of attacks.

First, let's count the number of rows for each attack type (and normal traffic). There are several ways to compute these counts.
Use the `print()` function to show the results of each method.

In [ ]:
# The first two are good if you want other stats besides count
# e.g. mean or standard deviation
type_counts = netattacks.groupby('type').count()
type_means = netattacks.groupby('type').mean()

# get a multi-index with multiple stats
type_counts = netattacks.groupby('type').agg(['count', 'mean'])

# cleanest for just counts
type_counts = netattacks['type'].value_counts()
type_counts.head()

We can see that `smurf` attacks were very common in this dataset. Normal traffic makes up a relatively small amount in this scenario.

# Plotting the data
In many cases, the distribution of the data is interesting. Does it follow a normal, bell curve distribution? Is it uniform? Multimodal?
Histograms are often an effective and quick way to see how a column is distributed.

In [ ]:
netattacks['duration'].hist()

This histogram is not particularly useful. It is likely affected by a few outliers. 

In [ ]:
netattacks['count'].hist()

# Correlations
The next basic analysis we will run is correlations. 

In [ ]:
netattacks['duration'].corr(netattacks['count'])

A negative correlation between two variables indicates that as the values 
of one variable increase, the values of the other decrease.  

# Reduce the problem
One way to approach the problem of identifying the specific origin of each type 
of traffic (i.e. `normal`, `smurf`, `satan`). This could be complex, and does 
not account for new types of attacks (this dataset was developed over 20 years 
ago--hackers have evolved a bit since then!).

A logical solution would be to reduce this to a binary problem. Normal
traffic is *good* and should be allowed on the network, and everything
else is *bad* and should be blocked by our firewall. We will not try to 
solve this problem in this lab (tune in next week for that!), but we 
will prepare the data for this by creating a new column that labels each
row as *good* or *bad*. 

In [ ]:
# https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
netattacks['label'] = np.where(netattacks['type'] == 'normal.', 'good', 'bad')
netattacks['label'].value_counts()

# Lab Exercises
To receive credit for this lab, please answer the following questions or complete the following tasks:
1. Can you run correlations between continuous and categorical variables? Why or why not?
2. Run a correlation between two or more other variables in the dataset. What might you interpret from this?
3. Summarize by protocol (`protocol_type`). Are there more *attacks* using `TCP` or `UDP`? (Hint: Group by two columns instead of one with `.group_by(['A', 'B'])`